In [ ]:
! pip install datasets evaluate transformers rouge-score nltk

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

In [ ]:
import transformers

print(transformers.__version__)

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from datasets import load_dataset
from evaluate import load

metric = load("rouge")

In [ ]:
import pandas as pd
from datasets import Dataset

# Charger le fichier CSV avec pandas
df = pd.read_csv("/kaggle/input/pfe-1-1/output1.csv")

In [ ]:

# Sélectionner les colonnes "cleaned_text" et "abstract"
df = df[['cleaned_text','shorter_abstract']]
# Afficher le DataFrame avec les colonnes sélectionnées
df

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2

train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)

In [ ]:
train_df

In [ ]:
# Filter rows where the type of 'shorter_abstract' column is not string
non_string_rows_index = train_df[~train_df['shorter_abstract'].apply(lambda x: isinstance(x, str))].index

# Drop the rows
train_df.drop(index=non_string_rows_index, inplace=True)

# Reset the index after dropping rows
train_df.reset_index(drop=True, inplace=True)
train_df

In [ ]:
test_df

In [ ]:
# Filter rows where the type of 'shorter_abstract' column is not string
non_string_rows_index = test_df[~test_df['shorter_abstract'].apply(lambda x: isinstance(x, str))].index

# Drop the rows
test_df.drop(index=non_string_rows_index, inplace=True)

# Reset the index after dropping rows
test_df.reset_index(drop=True, inplace=True)
test_df

In [ ]:
train_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming train_df is already defined
def word_counter(text):
    return len(text.split())

# Calculate the length of text in each column
train_df['cleaned_text_length'] = train_df['cleaned_text'].apply(word_counter)
train_df['shorter_abstract_length'] = train_df['shorter_abstract'].apply(word_counter)

# Get the maximum length in each column
max_cleaned_text_length = train_df['cleaned_text_length'].max()
max_shorter_abstract_length = train_df['shorter_abstract_length'].max()

In [ ]:
max_cleaned_text_length

In [ ]:
max_shorter_abstract_length

In [ ]:
# Plot the histogram for cleaned_text column
plt.figure(figsize=(10, 6))
plt.hist(train_df['cleaned_text_length'], bins=50, color='skyblue', edgecolor='black')
plt.title('Length Distribution of cleaned_text')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.axvline(x=max_cleaned_text_length, color='red', linestyle='--', label=f'Max Length: {max_cleaned_text_length}')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot the histogram for shorter_abstract column
plt.figure(figsize=(10, 6))
plt.hist(train_df['shorter_abstract_length'], bins=50, color='lightgreen', edgecolor='black')
plt.title('Length Distribution of shorter_abstract')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.axvline(x=max_shorter_abstract_length, color='red', linestyle='--', label=f'Max Length: {max_shorter_abstract_length}')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from datasets import Dataset

# Assuming you already have pandas DataFrames 'train_df' and 'test_df'

# Convert pandas DataFrames to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['cleaned_text','shorter_abstract']].head(20000))
test_dataset = Dataset.from_pandas(test_df.head(6000))


In [ ]:
from datasets import DatasetDict
dataset = DatasetDict({
    'train':train_dataset,
    'test':test_dataset
})
dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_input_length = 1024
max_target_length = 256

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["cleaned_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["shorter_abstract"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Charger le CSV dans un DataFrame pandas
df = pd.read_csv("/kaggle/input/pfe-1-1/output.csv")

# Limiter la taille de votre DataFrame (par exemple, aux 1000 premières lignes)
limited_df = df.head(1000)

# Convertir le DataFrame en Dataset
train_dataset = Dataset.from_pandas(limited_df)

# Définir une fonction de prétraitement
def preprocess_function(examples):
    # Prétraitement des exemples ici
    return examples

# Appliquer la fonction de prétraitement à l'ensemble de données
preprocessed_dataset = train_dataset.map(preprocess_function)

# Créer un DatasetDict contenant les ensembles d'entraînement et de test
dataset = DatasetDict({
    'train': preprocessed_dataset,
    'test': preprocessed_dataset,
})

# Exemple d'utilisation de la fonction preprocess_function
preprocess_function(dataset['train'][:2])



In [ ]:
dataset

In [ ]:
# Appliquer la fonction de prétraitement à l'ensemble de données
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Enregistrez le modèle checkpoint dans un répertoire spécifié
model.save_pretrained("/kaggle/working/t5-small-checkpoint")


In [ ]:
model_checkpoint = "/kaggle/working/t5-small-checkpoint"


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-pav1",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    save_total_limit=1,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    fp16_full_eval=False  # Désactiver la précision en demi-précision pour l'évaluation
)

In [ ]:
!nvcc --version


In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda

In [ ]:
import os

# Set DEBIAN_FRONTEND variable to avoid interactive prompts
os.environ['DEBIAN_FRONTEND'] = 'noninteractive'

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!sudo apt-get update -y
!sudo apt-get -y install cuda

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-pav1",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    save_total_limit=3,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    fp16_full_eval=False  # Désactiver la précision en demi-précision pour l'évaluation
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
tokenized_datasets

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
#aa93aa137a1a9d100aa3118a53d243fa4fe8543f

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

racha009/t5-small-checkpoint-finetuned-pav1